# Théorie apprentissage

Nous allons utiliser la base de données suivante : https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset#

C'est une base que nous avons vu dans le TP1 de DS,qui concerne le nombre de vélos prêter entre 2011 et 2012 par Capital Bikeshare à Whashington D.C.

# Partie 1

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

data = pd.read_csv('day.csv')

print(data.info())
display(data)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     731 non-null    int64  
 1   dteday      731 non-null    object 
 2   season      731 non-null    int64  
 3   yr          731 non-null    int64  
 4   mnth        731 non-null    int64  
 5   holiday     731 non-null    int64  
 6   weekday     731 non-null    int64  
 7   workingday  731 non-null    int64  
 8   weathersit  731 non-null    int64  
 9   temp        731 non-null    float64
 10  atemp       731 non-null    float64
 11  hum         731 non-null    float64
 12  windspeed   731 non-null    float64
 13  casual      731 non-null    int64  
 14  registered  731 non-null    int64  
 15  cnt         731 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.5+ KB
None


,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,2012-12-27,1,1,12,0,4,1,2,0.254167,0.226642,0.652917,0.350133,247,1867,2114
727,728,2012-12-28,1,1,12,0,5,1,2,0.253333,0.255046,0.590000,0.155471,644,2451,3095
728,729,2012-12-29,1,1,12,0,6,0,2,0.253333,0.242400,0.752917,0.124383,159,1182,1341
729,730,2012-12-30,1,1,12,0,0,0,1,0.255833,0.231700,0.483333,0.350754,364,1432,1796


In [10]:
Ydata = data['cnt'][:100] # on prend que les 100 premiers car sinon les algorithmes sont longs
Xdata = data.iloc[:100,4:13]

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(Xdata,Ydata, test_size = 0.3)

# Kernel Ridge

On implémente Ridge normal dans un premier temps.

In [11]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

meilleur_score = max(cross_val_score(Ridge(alpha = 50), X_train, Y_train, cv=5))
terme_regularisation = 1
                      
for i in range(0,50):
    if max(cross_val_score(Ridge(alpha = i), X_train, Y_train, cv=5)) > meilleur_score :
        terme_regularisation = i
        meilleur_score = max(cross_val_score(Ridge(alpha = i), X_train, Y_train, cv=5))

(terme_regularisation, meilleur_score)

(15, 0.7419434028166703)

In [12]:
def reg_Ridge(X_train, Y_train, terme_regularisation):
    # regresion linéaire Ridge : theta = (lamda*I + X'*X)^-1 * X' * Y
    I = np.identity(X_train.shape[1])
    C = (X_train.T).dot(X_train) + I * terme_regularisation
    Cinv = np.linalg.inv(C)
    theta = Cinv.dot(X_train.T)
    theta = theta.dot(Y_train)
    return theta


def prediction_ridge(X_test,theta):
    Y_pred = X_test.dot(theta)
    return Y_pred

Y_pred = prediction_ridge(X_test, reg_Ridge(X_train,Y_train, terme_regularisation))
#La constante de régularisation doit être déterminé par validation croissée, mais nous la fixons à 50 pour le moment
print('Erreur de prediction')
print(np.linalg.norm(Y_pred-Y_test))

Erreur de prediction
3224.2345416950157


In [13]:
from sklearn.linear_model import Ridge
ridge = Ridge()
ridge.fit(X_train,Y_train)
Y_pred = ridge.predict(X_test)
print('Erreur de prediction')
print(np.linalg.norm(Y_pred-Y_test))

Erreur de prediction
2431.968001056232


On implémente un kernel ridge avec le produit scalaire comme noyau dans un premier temps

In [14]:
def prod(x,y):
    (n,) = x.shape
    if n ==1 :
        return x.dot(y.T)
    else:
        return x.T.dot(y)

def kernel_Ridge(X_train, Y_train, terme_regularisation, kernel):
    # kernel Ridge : theta = (Gram + lamda*I)^-1 * Y
    I = np.identity(X_train.shape[0])
    Gram = np.identity(X_train.shape[0])
    for i in range(0, X_train.shape[0]): #on remplit la matrice de Gram
        for j in range(0, X_train.shape[0]):
            Gram[i][j] = kernel(X_train.iloc[i,:],X_train.iloc[j,:])
    C = Gram + I * terme_regularisation
    Cinv = np.linalg.inv(C)
    theta = Cinv.dot(Y_train)
    return theta
    
theta_kernel = kernel_Ridge(X_train, Y_train, terme_regularisation, lambda x, y: prod(x,y))

In [15]:
def prediction_kernel_ridge(X_train, X_test,theta, kernel): #f(X) = SUM(alpha_i * kernel(X,X_i))
    vect = np.zeros((X_test.shape[0], X_train.shape[0]))
    for i in range(0, X_test.shape[0]):
        for j in range(0, X_train.shape[0]):
            vect[i][j] = kernel(X_test.iloc[i,:],X_train.iloc[j,:])
    Y_pred = vect.dot(theta)
    return Y_pred

Y_pred_kernel = prediction_kernel_ridge(X_train, X_test, theta_kernel, lambda x, y: prod(x,y))
print('Erreur de prediction avec le noyau produit scalaire')
print(np.linalg.norm(Y_pred_kernel-Y_test))

Erreur de prediction avec le noyau produit scalaire
3224.2345416950147


Essayons de changer de noyau

In [16]:
import math

def poly(x,y):
    (n,) = x.shape
    if n ==1 :
        return (x.dot(y.T))**2
    else:
        return (x.T.dot(y))**2

theta_kernel_autre_noyau = kernel_Ridge(X_train, Y_train, terme_regularisation, lambda x, y: poly(x,y))

Y_pred_kernel = prediction_kernel_ridge(X_train, X_test, theta_kernel_autre_noyau, lambda x, y: poly(x,y))
print('Erreur de prediction avec le noyau polynomial')
print(np.linalg.norm(Y_pred_kernel-Y_test))

Erreur de prediction avec le noyau polynomial
4046.608640758396


In [17]:
def rbf(x,y):
    return math.exp(-0.01*(np.linalg.norm(x-y))**2)

theta_kernel_autre_noyau = kernel_Ridge(X_train, Y_train, terme_regularisation, lambda x, y: rbf(x,y))

Y_pred_kernel = prediction_kernel_ridge(X_train, X_test, theta_kernel_autre_noyau, lambda x, y: rbf(x,y))
print('Erreur de prediction avec le noyau rbf')
print(np.linalg.norm(Y_pred_kernel-Y_test))

Erreur de prediction avec le noyau rbf
4509.651438130885


Visiblement, le kernel polynomiale et rbf mène à du surapprentissage pour cette base de données. Les modèles sont trop complexes

# MKL

L'algorithme mettra quelques minutes à s'exécuter

In [18]:
def L2_norm_MKL(X_train, Y_train, X_test, epsilon,terme_regularisation,liste_kernel, n) :
    Fk = []
    beta = [1/len(liste_kernel) for i in range(len(liste_kernel))]
    alpha = 0
    for t in range(n):
        
        alpha_prime = alpha
        new_kernel = lambda x, y: beta[0] * liste_kernel[0](x,y) + beta[1] * liste_kernel[1](x,y) + beta[2] * liste_kernel[2](x,y)
        alpha = kernel_Ridge(X_train, Y_train, terme_regularisation, new_kernel)
        
        if np.linalg.norm(alpha - alpha_prime) < epsilon :
            return alpha, beta
        
        Fk = [prediction_kernel_ridge(X_train, X_test, alpha, lambda x, y: liste_kernel[i](x,y)) for i in range(len(liste_kernel))]
        
        for i in range(len(liste_kernel)):
            beta[i] = np.linalg.norm(Fk[i])**(2/3) / math.sqrt(np.linalg.norm(Fk[0]) ** (4/3) + np.linalg.norm(Fk[1]) ** (4/3) + np.linalg.norm(Fk[2]) ** (4/3))
    
    return alpha, beta

liste_kernel = [lambda x, y: prod(x,y), lambda x, y: poly(x,y), lambda x, y: rbf(x,y)]
alpha, beta = L2_norm_MKL(X_train, Y_train, X_test, 0.05,terme_regularisation, liste_kernel, 10)

alpha, beta
    
    
        

(array([ 37.91721187,   7.49371592,  28.01489967, -17.57842916,
         27.02293074,  -2.57144889,  25.39367606,  -7.50137824,
        -18.99545085, -12.48916562,   3.20282238,  -8.12635168,
         -5.9281379 , -22.19589725,  19.93501875, -25.67600551,
         27.79766261,   6.67023826,   9.57665496,  14.98693319,
         -3.9461173 ,  15.21781355,  18.28669763,  -9.0787792 ,
         15.51838958,  37.04043398,  14.84362344, -29.1281169 ,
         -4.21550685,   8.11309148, -10.88098716,  -7.92293181,
         25.67885926,  24.48024704,  12.08087527,   0.46370743,
        -14.60973032, -21.72501733,   2.91479948,  20.42448728,
         33.55258783,  35.24740556, -34.44623396, -67.24570735,
          4.77238588, -20.74234569,  13.6364869 ,  -9.86882779,
        -10.96939349,  -1.9399629 ,  -1.97927299,   2.90634328,
        -10.7033133 , -11.12434291, -29.98610797,  -3.06680866,
         23.25322478, -21.69655387, -10.47993807,   0.25030337,
          2.63461869,  21.37583824,  28.

In [19]:
new_kernel = lambda x, y: beta[0] * liste_kernel[0](x,y) + beta[1] * liste_kernel[1](x,y) + beta[2] * liste_kernel[2](x,y)
Y_pred_kernel = prediction_kernel_ridge(X_train, X_test, alpha, new_kernel)

print('Erreur de prediction kernel MKL')
print(np.linalg.norm(Y_pred_kernel-Y_test))

Erreur de prediction kernel MKL
3202.2479041789525


# Combinaison Uniforme

In [20]:
kernel_uniforme = lambda x, y: (1/3) * liste_kernel[0](x,y) + (1/3) * liste_kernel[1](x,y) + (1/3)  * liste_kernel[2](x,y)
theta_kernel_uniforme = kernel_Ridge(X_train, Y_train, terme_regularisation, kernel_uniforme)
Y_pred_kernel = prediction_kernel_ridge(X_train, X_test, theta_kernel_uniforme, kernel_uniforme)

print('Erreur de prediction kernel uniforme')
print(np.linalg.norm(Y_pred_kernel-Y_test))

Erreur de prediction kernel uniforme
3356.7057228461586


Les meilleurs résultats sont obtenus avec la méthode MKL, suivi de la regression Ridge avec un kernel produit scalaire et enfin la combinaison uniforme

# Partie 2

# Random Kitchen Sinks

In [21]:
# %Training
def Random_feature(X_train, Y_train, X_test, terme_regularisation):
    W = np.random.normal(0,0.2,(X_train.shape[0],X_train.shape[1]))
    phi = np.concatenate([np.cos(X_train.dot(W.T)), np.sin(X_train.dot(W.T))],axis = 1)
    I = np.identity(X_train.shape[0])
    C = phi.dot(phi.T) + I * terme_regularisation
    Cinv = np.linalg.pinv(C)
    theta = Cinv.dot(Y_train)
    
# %Testing

    phi_test = np.concatenate([np.cos(X_test.dot(W.T)), np.sin(X_test.dot(W.T))],axis = 1)
    K = phi_test.dot(phi.T)
    return K.dot(theta)

Y_pred_random = Random_feature(X_train, Y_train, X_test, terme_regularisation)
   

In [22]:
print('Erreur de prediction Random Kitchen Sinks')
print(np.linalg.norm(Y_pred_random-Y_test))


Erreur de prediction Random Kitchen Sinks
2604.7711217395704


# Methode de Nystrom

In [23]:
m = int(X_train.shape[0] / 3)

def Nystrom(X_train, kernel, m):
    np.random.randint(low=1, high=X_train.shape[0], size=m)
    C = np.zeros((X_train.shape[0], m))
    for i in range(0, X_train.shape[0]):
        for j in range(0,m):
            C[i][j] = kernel(X_train.iloc[i,:],X_train.iloc[j,:])

    W = np.zeros((m, m))
    for i in range(0, m):
        for j in range(0,m):
            W[i][j] = kernel(X_train.iloc[i,:],X_train.iloc[j,:])

    Winv = np.linalg.pinv(W)
    G_nystrom = Winv.dot(C.T)
    G_nystrom = C.dot(G_nystrom)

    return G_nystrom

G_nystrom = Nystrom(X_train, lambda x, y: prod(x,y), m)
G_nystrom.shape

(70, 70)

In [24]:
def kernel_Ridge_avec_Nystrom(X_train, Y_train, terme_regularisation, kernel, Gram):
    # kernel Ridge : theta = (Gram + lamda*I)^-1 * Y
    I = np.identity(X_train.shape[0])
    C = Gram + I * terme_regularisation
    Cinv = np.linalg.inv(C)
    theta = Cinv.dot(Y_train)
    return theta

theta_nystrom = kernel_Ridge_avec_Nystrom(X_train, Y_train, terme_regularisation, lambda x, y: prod(x,y), Nystrom(X_train, lambda x, y: prod(x,y), m))

In [25]:
Y_pred_nystrom = prediction_kernel_ridge(X_train, X_test, theta_nystrom, lambda x, y: prod(x,y))
print('Erreur de prediction avec Nystrom')
print(np.linalg.norm(Y_pred_nystrom-Y_test))

Erreur de prediction avec Nystrom
3224.234541699633


L'approximation de Nyström est meilleure dans notre cas.